In [ ]:
import pandas as pd

df = pd.read_json("10000_All_Beauty.json.xz")

In [ ]:
from transformers import pipeline
zeroshot_classifier = pipeline("zero-shot-classification", 
                               model="MoritzLaurer/ModernBERT-large-zeroshot-v2.0", 
                               device_map="auto")

In [ ]:
hypothesis_template = "This text has a {} sentiment"
classes = ["positive", "negative"]

In [ ]:
zeroshot_classifier(df.iloc[0]["text"], classes, hypothesis_template=hypothesis_template, multi_label=False)

In [ ]:
from tqdm.auto import tqdm
df["positive"] = None
df["negative"] = None
for i, r in tqdm(df.iterrows(), total=len(df)):
    o = zeroshot_classifier(r["text"], classes, hypothesis_template=hypothesis_template, multi_label=False)
    df.at[i, o["labels"][0]] = o["scores"][0]
    df.at[i, o["labels"][1]] = o["scores"][1]

In [ ]:
df.sample(10, random_state=42)

In [ ]:
df["zeroshot_rating"] = None
for i, r in tqdm(df.iterrows(), total=len(df)):
    if r["positive"] > 5*r["negative"]:
        df.at[i, "zeroshot_rating"] = 5
    else:
        df.at[i, "zeroshot_rating"] = 1        

In [ ]:
wrong = df[df["rating"] != df["zeroshot_rating"]]
wrong

In [ ]:
1-len(wrong)/len(df)

In [ ]:
# You might think that it is much faster to run this in batch mode:

In [ ]:
zeroshot_classifier(list(df.iloc[0:10]["text"]), classes, hypothesis_template=hypothesis_template, multi_label=False)

In [ ]:
# But it's not...

In [ ]:
from tqdm.auto import trange
MAX = 100
for s in trange(0, len(df), MAX):
    o = zeroshot_classifier(list(df[s:s+MAX]["text"]), classes, hypothesis_template=hypothesis_template, multi_label=False)
    #df.at[i, o["labels"][0]] = o["scores"][0]
    #df.at[i, o["labels"][1]] = o["scores"][1]